In [ ]:
import os

if os.name == 'nt':
    measuring_root = "D:/measuring"
else:
    measuring_root = "/Users/jesse/Thesis/Code"

execfile(os.path.join(measuring_root, "analysis/scripts/setup_analysis.py"))
import analysis.lib.purification.purify_delayfb as pu_delayfb
reload(pu_delayfb)
import analysis.lib.fastcarboncontrol.fcc as fcc
reload(fcc)
import matplotlib as mpl
mpl.style.use('seaborn-bright')
%matplotlib inline

In [ ]:
reload(pu_delayfb)

C2_ot = "20170712161100"
C4_ot = "20170712161900"

c2_x, c2_y, c2_y_u, c2_decay_fit = pu_delayfb.number_of_repetitions(
    contains = '_X',
    older_than=C2_ot,
    do_fit = True,
    fixed=[0,2,5,6,4],
    ret_data_fit=True
)

c4_x, c4_y, c4_y_u, c4_decay_fit = pu_delayfb.number_of_repetitions(
    contains = '_X',
    older_than=C4_ot,
    do_fit = True,
    fixed=[0,2,5,6,4],
    ret_data_fit=True
)

# N_exp_c2 = c2_decay_fit['params_dict']['T']
# N_exp_c4 = c4_decay_fit['params_dict']['T']

In [ ]:
plt.figure()
fit_xvals = np.linspace(0.,600.,1000)

plt.errorbar(
    c2_x, c2_y, c2_y_u, fmt='o', color='C0',
#     label = r"C2 measured"
)
plt.plot(
    fit_xvals, c2_decay_fit['fitfunc'](fit_xvals), color='C0', 
    label = r'C2, $N_{1/e} = %d$' % np.round(c2_decay_fit['params_dict']['T'])
)

plt.errorbar(
    c4_x, c4_y, c4_y_u, fmt='o', color='C2',
#     label = r"C4 measured"
)
plt.plot(
    fit_xvals, c4_decay_fit['fitfunc'](fit_xvals), color='C2',
    label = r'C4, $N_{1/e} = %d$' % np.round(c4_decay_fit['params_dict']['T'])
)

plt.xlabel("Number of LDE attempts")
plt.ylabel("Bloch vector length")

plt.legend()

# Feedback tomographies

In [ ]:
reload(pu_delayfb)

tomo_ots = dict()

tomo_ots['XI'] = [
    "20170712163000",
    "20170712163800",
]
tomo_ots['IX'] = [
    "20170712164700",
    "20170712165500",
]
tomo_ots['XX'] = [
    "20170712170500",
    "20170712171200",
    "20170712172000"
]
tomo_ots['XX_DES'] = [
    "20170712173800",
    "20170712174400",
]
tomo_ots['XX_DPS'] = [
    "20170712175200",
    "20170712180000",
    "20170712181000"
]

# for ot in data_older_thans:
#     pu_ssro.number_of_repetitions(contains = 'ADwin_phase_compensation',
#                                   older_than=ot,
#                                   do_fit = True,
#                                   fixed=[0,2,5,6],
#                                   x_only=True)

tomo_data_x = dict()
tomo_data_y = dict()
tomo_data_y_u = dict()
tomo_data_fr = dict()

for tomo in tomo_ots.iterkeys():
    print "Tomo: " + tomo
    tomo_data_x[tomo], tomo_data_y[tomo], tomo_data_y_u[tomo], tomo_data_fr[tomo] = \
    pu_delayfb.number_of_repetitions_stitched(
        contains='phase_fb_delayline', 
        older_thans=tomo_ots[tomo], 
        do_fit=True, 
        x_only=True,
        fit_n=1.0,
        fixed=[0,2,5,6,4],
        ret_data_fit=True)

# Combined plots

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
plt.xlabel("Number of LDE attempts")
plt.ylabel(r"Expectation value")

tomos = ['XI', 'IX', 'XX']

fit_xvals = np.linspace(0.,1000., 2000.)

for i in range(len(tomos)):
    tomo = tomos[i]
    plt.errorbar(
        tomo_data_x[tomo], tomo_data_y[tomo], tomo_data_y_u[tomo], 
        fmt='o', color="C%d" % i, 
        label = r"$\langle %s \rangle$, $N_{1/e} = %d$" % (tomo, np.round(tomo_data_fr[tomo]['params_dict']['T']))
    )
    plt.plot(fit_xvals, tomo_data_fr[tomo]['fitfunc'](fit_xvals))

plt.legend()
plt.xlim(0,1000)
plt.ylim(-0.1,1)

plt.savefig("multi_C_state_preservation.png", dpi=300)

N_exp_c2 = tomo_data_fr['XI']['params_dict']['T']
N_exp_c4 = tomo_data_fr['IX']['params_dict']['T']

A_c2 = tomo_data_fr['XI']['params_dict']['A']
A_c4 = tomo_data_fr['IX']['params_dict']['A']

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
plt.xlabel("Number of LDE attempts")
plt.ylabel(r"Expectation value")

tomos = ['XX', 'XX_DES', 'XX_DPS']
labels = [r'$\langle XX \rangle$', r'$\langle X+45\degree, X+45\degree \rangle$', r'$\langle X+45\degree, X-45\degree \rangle$']

fit_xvals = np.linspace(0.,1000., 2000.)

for i in range(len(tomos)):
    tomo = tomos[i]
    plt.errorbar(
        tomo_data_x[tomo], tomo_data_y[tomo], tomo_data_y_u[tomo], 
        fmt='o', color="C%d" % i, 
        label = labels[i] + r", $N_{1/e} = %d$" % (np.round(tomo_data_fr[tomo]['params_dict']['T']))
    )
    plt.plot(fit_xvals, tomo_data_fr[tomo]['fitfunc'](fit_xvals))

plt.legend()
plt.xlim(0,1000)
plt.ylim(-0.1,1)

plt.savefig("multi_C_classical_des_dps.png", dpi=300)

# Detuned measurements

In [ ]:
reload(pu_delayfb)

det_ots = dict()

det_ots['XX'] = [
    '20170712143900',
    '20170712145000',
]

det_ots['XI'] = [
    '20170712145700',
    '20170712150300'
]

det_ots['IX'] = [
    '20170712150900',
    '20170712151500',
]

det_ots['XX_det_8_4'] = [
    '20170712152900',
    '20170712153600',
]

det_ots['XX_det_8_16'] = [
    '20170712154300',
    '20170712154900'
]

det_ots['XX_det_1_0'] = [
    '20170714111000',
    '20170714111800',
    '20170714112500'
]

det_data_x = dict()
det_data_y = dict()
det_data_y_u = dict()
det_data_fr = dict()

for tomo in det_ots.iterkeys():
    print "Tomo: " + tomo
    fit_guess = {
        'fit_f': 8./360.
    }
    det_data_x[tomo], det_data_y[tomo], det_data_y_u[tomo], det_data_fr[tomo] = \
    pu_delayfb.number_of_repetitions_stitched(
        contains='phase_fb_delayline', 
        older_thans=det_ots[tomo], 
        do_fit=True, 
        x_only=True,
        fixed=[0,2,4],
        ret_data_fit=True,
        tomo_basis='XX' if (tomo == 'XX_det_1_0') else 'X',
        **fit_guess
    )

In [ ]:
def fit_double_decaying_cos(g_f1, g_A1, g_phi1, g_t1, g_f2, g_A2, g_phi2, g_t2, g_o ,*arg):
    ''' quite a specific function, for electron nuclear control, maybe place somewhere else '''
    fitfunc_str = '''(A1 *exp(-x/t1) cos(2pi * (f1*x + phi1/360) ) + a1)*
                     (A2 *exp(-x/t2) cos(2pi * (f2*x + phi2/360) ) + a2)/2+ o '''

    f1 = fit.Parameter(g_f1, 'f1')
    #a1 = fit.Parameter(g_a1, 'a1')
    A1 = fit.Parameter(g_A1, 'A1')
    phi1 = fit.Parameter(g_phi1, 'phi1')
    t1   = fit.Parameter(g_t1, 't1')

    f2 = fit.Parameter(g_f2, 'f2')
    #a2 = fit.Parameter(g_a2, 'a2')
    A2 = fit.Parameter(g_A2, 'A2')
    phi2 = fit.Parameter(g_phi2, 'phi2')
    t2   = fit.Parameter(g_t2, 't2')
    o = fit.Parameter(g_o, 'o')

    #p0 = [f1, a1, A1, phi1, t1, f2, a2, A2, phi2, t2]
    p0 = [f1, A1, phi1, t1, f2, A2, phi2, t2,o]

    def fitfunc(x):
        return (A1()*np.exp(-x/t1()) * np.cos(2*np.pi*( f1()*x + phi1()/360.)))*(A2()*np.exp(-x/t2()) * np.cos(2*np.pi*( f2()*x + phi2()/360.)))+o()

    return p0, fitfunc, fitfunc_str

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)

tomo = 'XX'

plt.errorbar(
        det_data_x[tomo], det_data_y[tomo], det_data_y_u[tomo], 
        fmt='o', color="C0"
    )

p0, fitfunc, fitfunc_str = fit_double_decaying_cos(
    0.022508, 0.8, 10.0, np.inf, #det_data_fr['XI']['params_dict']['T'],
    0.022199, 0.8, -20.0, np.inf, #det_data_fr['IX']['params_dict']['T'],
    0.0
)

fit_xvals = np.linspace(0.,70., 1000.)

fixed = [0,4,8,3,7]

x = det_data_x[tomo]
y = det_data_y[tomo]

ax.plot(fit_xvals, fitfunc(fit_xvals), ':', lw=2)

fit_result = fit.fit1d(x, y, None, p0=p0, fitfunc=fitfunc, do_print=True, fixed=fixed, ret=True)

plt.plot(fit_xvals, fit_result['fitfunc'](fit_xvals))

plt.ylim(-1.0, 1.0)

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
plt.xlabel("Number of LDE attempts")
plt.ylabel(r"Expectation value")

tomos = ['XI', 'IX', 'XX']

fit_xvals = np.linspace(0.,70., 1000.)

for i in range(len(tomos)):
    tomo = tomos[i]
    label = r'$\langle XX \rangle$' if tomo == 'XX' else None
    plt.errorbar(
        det_data_x[tomo], det_data_y[tomo], det_data_y_u[tomo], 
        fmt='o', color="C%d" % i,
        label=label
    )
    if tomo != 'XX':
        plt.plot(fit_xvals, det_data_fr[tomo]['fitfunc'](fit_xvals), label = r"$\langle %s \rangle$" % (tomo))

plt.plot(
    fit_xvals, 
    1*det_data_fr['XI']['fitfunc'](fit_xvals) * det_data_fr['IX']['fitfunc'](fit_xvals),
    color='C3',
    label = r"$\langle XI \rangle \cdot \langle IX \rangle$"
)

# plt.plot(fit_xvals, fit_result['fitfunc'](fit_xvals), color='C2', label=r"$\langle XX \rangle$")

plt.legend(loc=3)
plt.xlim(0,70)
plt.ylim(-1,1)

plt.savefig("IX_XI_XX_detuned_clean_msmt.png", dpi=300)

In [ ]:
det_data_fr['XI']['params_dict']['T']

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)

tomo = 'XX_det_8_4'

plt.errorbar(
        det_data_x[tomo], det_data_y[tomo], det_data_y_u[tomo], 
        fmt='o', color="C0"
    )

p0, fitfunc, fitfunc_str = fit_double_decaying_cos(
    8./360., 0.8, 0.0, det_data_fr['XI']['params_dict']['T'],
    4./360., 0.8, 0.0, det_data_fr['IX']['params_dict']['T'],
    0.0
)

fit_xvals = np.linspace(0.,70., 1000.)

fixed = [8]

x = det_data_x[tomo]
y = det_data_y[tomo]

ax.plot(fit_xvals, fitfunc(fit_xvals), ':', lw=2)

fit_result = fit.fit1d(x, y, None, p0=p0, fitfunc=fitfunc, do_print=True, fixed=fixed, ret=True)

plt.plot(fit_xvals, fit_result['fitfunc'](fit_xvals))

plt.ylim(-1.0, 1.0)

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)

tomo = 'XX_det_8_16'

plt.errorbar(
        det_data_x[tomo], det_data_y[tomo], det_data_y_u[tomo], 
        fmt='o', color="C0"
    )

p0, fitfunc, fitfunc_str = fit_double_decaying_cos(
    8./360., 0.9, 0.0, det_data_fr['XI']['params_dict']['T'],
    16./360., 0.9, 0.0, det_data_fr['IX']['params_dict']['T'],
    0.0
)

fit_xvals = np.linspace(0.,70., 1000.)

fixed = [8,3,7]

x = det_data_x[tomo]
y = det_data_y[tomo]

ax.plot(fit_xvals, fitfunc(fit_xvals), ':', lw=2)

fit_result = fit.fit1d(x, y, None, p0=p0, fitfunc=fitfunc, do_print=True, fixed=fixed, ret=True)

plt.plot(fit_xvals, fit_result['fitfunc'](fit_xvals))

plt.ylim(-1.0, 1.0)

# Phase noise fits

In [ ]:
omega_2_ms_0 = 442998.53 * 2*np.pi
omega_2_ms_m1 = 475418.21 * 2*np.pi

d_omega_2 = omega_2_ms_0 - omega_2_ms_m1

omega_4_ms_0 = 442804.81 * 2*np.pi
omega_4_ms_m1 = 416208.47 * 2*np.pi

d_omega_4 = omega_4_ms_0 - omega_4_ms_m1

In [ ]:
from analysis.lib.fitting import fit
from analysis.lib.tools import plot

def fit_cosine_centred_gamma(g_omega, g_tau, g_phi0):
    fitfunc_str = "Re[ exp(i * (phi0 - omega*x*tau)) * (1 - i*omega*tau)**-x ]"
    
    omega = fit.Parameter(g_omega, 'omega')
    tau = fit.Parameter(g_tau, 'tau')
    phi0 = fit.Parameter(g_phi0, 'phi0')
    
    p0 = [omega, tau, phi0]
    
    def fitfunc(x):
        return np.real(np.exp(1j*(phi0() - omega()*x*tau())) * np.power(1-1j*omega()*tau(), -x))
    
    return p0, fitfunc, fitfunc_str

fixed=[0,2]
show_guess = True

for x, y, y_u, d_omega_c in [[c2_x, c2_y, c2_y_u, d_omega_2], [c4_x, c4_y, c4_y_u, d_omega_4]]:

#     x, y, y_u = c2_x, c2_y, c2_y_u
    plt.figure()

    p0, fitfunc, fitfunc_str = fit_cosine_centred_gamma(d_omega_c, 
                                                        600e-9, 0.0)

    if show_guess:
        # print decay
        plt.plot(np.linspace(x[0], x[-1], 201), fitfunc(np.linspace(x[0], x[-1], 201)), ':', lw=2)

    fit_result = fit.fit1d(x, y, None, p0=p0, fitfunc=fitfunc, do_print=True, ret=True, VERBOSE=True, fixed=fixed)
    print("Tau: %.2f +/- %.2f ns" % (fit_result['params_dict']['tau'] * 1e9, fit_result['error_dict']['tau'] * 1e9))

    plt.errorbar(x,y,y_u, fmt='o')

    plot.plot_fit1d(fit_result, np.linspace(x[0], x[-1], 100), ax=plt.gca(), plot_data=False)
    plt.show()

In [ ]:
print("C2 decay results")
sigma_phi_c2_decay = np.sqrt(2./N_exp_c2)
sigma_t_c2_decay = sigma_phi_c2_decay / d_omega_2

print("sigma-phi1: %.2f deg" % (sigma_phi_c2_decay / np.pi * 180.))
print("sigma-t: %.2f ns" % (sigma_t_c2_decay * 1e9))

print("C4 decay results")
sigma_phi_c4_decay = np.sqrt(2./N_exp_c4)
sigma_t_c4_decay = sigma_phi_c4_decay / d_omega_4
print("sigma-phi1: %.2f deg" % (sigma_phi_c4_decay / np.pi * 180.))
print("sigma-t: %.2f ns" % (sigma_t_c4_decay * 1e9))

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)

Ns = np.linspace(0,70,1000.0)

sigma_deltat = 650e-9

detuning_2 = 8.0
detuning_4 = 4.0

fcc.plot_noisey_carbon_correlations(Ns, sigma_deltat, A_c2, A_c4, d_omega_2, d_omega_4, detuning_2, detuning_4, label="noisy model")
fcc.plot_noisey_carbon_correlations(Ns, 0.0, A_c2, A_c4, d_omega_2, d_omega_4, detuning_2, detuning_4, label="noise free model")

# plt.plot(Ns, p_term, label="+ term")
# plt.plot(Ns, m_term, label="- term")

tomo = 'XX_det_8_4'

plt.errorbar(
        det_data_x[tomo], det_data_y[tomo], det_data_y_u[tomo], 
        fmt='o', color="C0"
    )


plt.xlabel("LDE attempts")
plt.ylabel(r"$\langle X_2 X_4 \rangle$")
# plt.title(r"$\phi_2 = %.1f \degree$, $\phi_4 = %.1f \degree$" % (phi_2_deg, phi_4_deg))
plt.title(r"$\Delta\phi_2 = %.1f \degree$, $\Delta\phi_4 = %.1f \degree$" % (detuning_2, detuning_4))
plt.legend()
plt.ylim(-1.0, 1.0)

plt.savefig("noisey-xx-det-8-4.png", dpi=300)

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)

Ns = np.linspace(0,70,1000.0)

sigma_deltat = 650e-9

detuning_2 = 8.0
detuning_4 = 16.0

fcc.plot_noisey_carbon_correlations(Ns, sigma_deltat, A_c2, A_c4, d_omega_2, d_omega_4, detuning_2, detuning_4, label="noisy model")
fcc.plot_noisey_carbon_correlations(Ns, 0.0, A_c2, A_c4, d_omega_2, d_omega_4, detuning_2, detuning_4, label="noise free model")

# plt.plot(Ns, p_term, label="+ term")
# plt.plot(Ns, m_term, label="- term")

tomo = 'XX_det_8_16'

plt.errorbar(
        det_data_x[tomo], det_data_y[tomo], det_data_y_u[tomo], 
        fmt='o', color="C0"
    )


plt.xlabel("LDE attempts")
plt.ylabel(r"$\langle X_2 X_4 \rangle$")
# plt.title(r"$\phi_2 = %.1f \degree$, $\phi_4 = %.1f \degree$" % (phi_2_deg, phi_4_deg))
plt.title(r"$\Delta\phi_2 = %.1f \degree$, $\Delta\phi_4 = %.1f \degree$" % (detuning_2, detuning_4))
plt.legend(loc=3)
plt.ylim(-1.0, 1.0)

plt.savefig("noisey-xx-det-8-16.png", dpi=300)

In [ ]:
reload(fcc)

fig = plt.figure()
ax = plt.subplot(111)

Ns = np.linspace(0,70,1000.0)

sigma_deltat = 650e-9

detuning_2 = 8.0
detuning_4 = 8.0

fcc.plot_noisey_carbon_correlations(Ns, sigma_deltat, A_c2, A_c4, d_omega_2, d_omega_4, detuning_2, detuning_4, label="noisy model")
fcc.plot_noisey_carbon_correlations(Ns, 0.0, A_c2, A_c4, d_omega_2, d_omega_4, detuning_2, detuning_4, label="noise free model")

# plt.plot(Ns, p_term, label="+ term")
# plt.plot(Ns, m_term, label="- term")

tomo = 'XX'

plt.errorbar(
        det_data_x[tomo], det_data_y[tomo], det_data_y_u[tomo], 
        fmt='o', color="C0"
    )


plt.xlabel("LDE attempts")
plt.ylabel(r"$\langle X_2 X_4 \rangle$")
plt.title(r"$\Delta\phi_2 = %.1f \degree$, $\Delta\phi_4 = %.1f \degree$" % (detuning_2, detuning_4))
plt.legend()
plt.ylim(-1.0, 1.0)
plt.savefig("noisey-xx-det-8-8.png", dpi=300)

In [ ]:
reload(fcc)

fig = plt.figure()
ax = plt.subplot(111)

Ns = np.linspace(0,1000.0,1000.0)

sigma_deltat = 650e-9

detuning_2 = 1.0
detuning_4 = 0.0

fcc.plot_noisey_carbon_correlations(Ns, sigma_deltat, A_c2, A_c4, d_omega_2, d_omega_4, detuning_2, detuning_4, label="noisey model")
fcc.plot_noisey_carbon_correlations(Ns, 0.0, A_c2, A_c4, d_omega_2, d_omega_4, detuning_2, detuning_4, label="noise free model")

# plt.plot(Ns, p_term, label="+ term")
# plt.plot(Ns, m_term, label="- term")

tomo = 'XX_det_1_0'

plt.errorbar(
        det_data_x[tomo], det_data_y[tomo], det_data_y_u[tomo], 
        fmt='o', color="C0"
    )


plt.xlabel("LDE attempts")
plt.ylabel(r"$\langle X_2 X_4 \rangle$")
plt.title(r"$\Delta\phi_2 = %.1f \degree$, $\Delta\phi_4 = %.1f \degree$" % (detuning_2, detuning_4))
plt.legend()
plt.ylim(-1.0, 1.0)
plt.savefig("noisey-xx-det-1-0.png", dpi=300)